# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

vacation_df




,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed
0,2014369,Ulety,10,RU,86,51.3572,112.4849,13.56,1.92
1,1636308,Manokwari,99,ID,72,-0.8667,134.0833,26.50,3.13
2,3874787,Punta Arenas,75,CL,61,-53.1500,-70.9167,7.96,10.80
3,4035715,Avarua,75,CK,83,-21.2078,-159.7750,24.03,6.17
4,5106841,Albany,14,US,54,42.6001,-73.9662,24.46,0.45
...,...,...,...,...,...,...,...,...,...
584,1150965,Ranong,98,TH,89,9.9658,98.6348,26.53,4.39
585,400666,Rhodes,0,GR,81,36.4408,28.2225,26.55,8.75
586,1855540,Naze,100,JP,87,28.3667,129.4833,26.51,11.86
587,293842,Kiryat Gat,0,IL,100,31.6100,34.7642,27.72,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
fig=gmaps.figure()

heat=gmaps.heatmap_layer(vacation_df[["Lat","Lng"]],vacation_df["Humidity"])

fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:


weather_con_df = vacation_df.loc[(vacation_df["Temperature"] >=20) & (vacation_df["Temperature"] <=35) & (vacation_df["Wind Speed"] <=6) & (vacation_df["Cloudiness"] <=3) & (vacation_df["Humidity"] <=30)]
hotel_df = weather_con_df
hotel_df



,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed
11,214389,Kaniama,2,CD,27,-7.5667,24.1833,26.74,1.43
82,296895,Yerköy,0,TR,27,39.6381,34.4672,26.54,3.20
164,3460087,Jardim,2,BR,24,-21.4803,-56.1381,26.77,1.63
231,895953,Zambezi,2,ZM,23,-13.5432,23.1047,20.23,2.07
309,6945979,Lillooet,0,CA,27,50.6856,-121.9420,28.46,0.03
395,315621,Emirdağ,0,TR,30,39.0197,31.1500,25.27,2.54
407,5703673,Elko,0,US,24,41.0002,-115.5012,31.11,2.24
435,3460531,Ituaçu,3,BR,19,-13.8133,-41.2967,31.67,3.00
505,3472344,América Dourada,2,BR,18,-11.4553,-41.4361,32.84,2.79
513,3471368,Astorga,1,BR,20,-23.2325,-51.6656,21.13,2.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
for index, row in hotel_df.iterrows():
    
    print(index)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    lat = row["Lat"]
    lng = row["Lng"]    
        
    params={"location": f"{lat},{lng}",
                "types": "lodging",
                "radius": 5000,
                "key": g_key,
                }

        
    params["location"] = f"{lat}, {lng}"
        
    response = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]

    except (KeyError, IndexError):
        
        hotel_df.loc[index,"Hotel Name"] = "No Data"
    
hotel_df


11
82
164
231
309
395
407
435
505
513


,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed,Hotel Name
11,214389,Kaniama,2,CD,27,-7.5667,24.1833,26.74,1.43,No Data
82,296895,Yerköy,0,TR,27,39.6381,34.4672,26.54,3.20,Yerköy Öğretmenevi
164,3460087,Jardim,2,BR,24,-21.4803,-56.1381,26.77,1.63,Estancia Hotel - Jardim MS
231,895953,Zambezi,2,ZM,23,-13.5432,23.1047,20.23,2.07,Royal Kutachika Lodge
309,6945979,Lillooet,0,CA,27,50.6856,-121.9420,28.46,0.03,Hotel Victoria
395,315621,Emirdağ,0,TR,30,39.0197,31.1500,25.27,2.54,Emirdağ Öğretmenevi
407,5703673,Elko,0,US,24,41.0002,-115.5012,31.11,2.24,No Data
435,3460531,Ituaçu,3,BR,19,-13.8133,-41.2967,31.67,3.00,Hotel Alelos
505,3472344,América Dourada,2,BR,18,-11.4553,-41.4361,32.84,2.79,Pousada do Ninho
513,3471368,Astorga,1,BR,20,-23.2325,-51.6656,21.13,2.11,Rancho Jads


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Kaniama</dd>\n<dt>Country</dt><dd>CD</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Yerköy Öğretmenevi</dd>\n<dt>City</dt><dd>Yerköy</dd>\n<dt>Country</dt><dd>TR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Estancia Hotel - Jardim MS</dd>\n<dt>City</dt><dd>Jardim</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Royal Kutachika Lodge</dd>\n<dt>City</dt><dd>Zambezi</dd>\n<dt>Country</dt><dd>ZM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Victoria</dd>\n<dt>City</dt><dd>Lillooet</dd>\n<dt>Country</dt><dd>CA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Emirdağ Öğretmenevi</dd>\n<dt>City</dt><dd>Emirdağ</dd>\n<dt>Country</dt><dd>TR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Elko</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Alelos</dd>\n<dt>City</dt><dd>Ituaçu</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Pousada do Ninho</dd>\

In [18]:
type(locations)

pandas.core.frame.DataFrame

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig